In [1]:
import sqlite3
import json

In [2]:
con = sqlite3.connect("db_20191123-20221018.sqlite")
con.execute("ATTACH 'db_20200825-20211226.sqlite' as db_2")
con.execute("ATTACH 'db_20220104-20230518.sqlite' as db_3")
con.row_factory = sqlite3.Row
cur = con.cursor()


In [3]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cur.fetchall())
cur.execute("SELECT name FROM db_2.sqlite_master")
print(cur.fetchall())
cur.execute("SELECT name FROM db_3.sqlite_master")
print(cur.fetchall())

[<sqlite3.Row object at 0x000002CC88CFBFA0>, <sqlite3.Row object at 0x000002CC88CF9C90>, <sqlite3.Row object at 0x000002CC88CFAF80>, <sqlite3.Row object at 0x000002CC88CFB730>, <sqlite3.Row object at 0x000002CC88CFAF20>, <sqlite3.Row object at 0x000002CC88CFBCD0>, <sqlite3.Row object at 0x000002CC88CFBA90>, <sqlite3.Row object at 0x000002CC88CF8A30>, <sqlite3.Row object at 0x000002CC88CFB880>, <sqlite3.Row object at 0x000002CC88CFB3A0>, <sqlite3.Row object at 0x000002CC88CFB250>, <sqlite3.Row object at 0x000002CC88CFAA40>, <sqlite3.Row object at 0x000002CC88CFB430>, <sqlite3.Row object at 0x000002CC88CF8040>, <sqlite3.Row object at 0x000002CC88CFBD60>, <sqlite3.Row object at 0x000002CC88CF8820>, <sqlite3.Row object at 0x000002CC88CF8070>, <sqlite3.Row object at 0x000002CC88CF82E0>, <sqlite3.Row object at 0x000002CC88CFB400>, <sqlite3.Row object at 0x000002CC88CF8A00>, <sqlite3.Row object at 0x000002CC88CF8AC0>, <sqlite3.Row object at 0x000002CC88CF9AB0>, <sqlite3.Row object at 0x000002

In [4]:
query = """
    SELECT * from messages, db_2.messages, db_3.messages
    """
cur.execute(query)
print(cur.fetchone().keys())

['rowid', 'id', 'json', 'readStatus', 'expires_at', 'sent_at', 'schemaVersion', 'conversationId', 'received_at', 'source', 'hasAttachments', 'hasFileAttachments', 'hasVisualMediaAttachments', 'expireTimer', 'expirationStartTimestamp', 'type', 'body', 'messageTimer', 'messageTimerStart', 'messageTimerExpiresAt', 'isErased', 'isViewOnce', 'sourceUuid', 'serverGuid', 'sourceDevice', 'storyId', 'isStory', 'isChangeCreatedByUs', 'isTimerChangeFromSync', 'isGroupLeaveEvent', 'isGroupLeaveEventFromOther', 'seenStatus', 'storyDistributionListId', 'callId', 'callMode', 'expiresAt', 'shouldAffectActivity', 'shouldAffectPreview', 'isUserInitiatedMessage', 'rowid', 'id', 'json', 'readStatus', 'expires_at', 'sent_at', 'schemaVersion', 'conversationId', 'received_at', 'source', 'deprecatedSourceDevice', 'hasAttachments', 'hasFileAttachments', 'hasVisualMediaAttachments', 'expireTimer', 'expirationStartTimestamp', 'type', 'body', 'messageTimer', 'messageTimerStart', 'messageTimerExpiresAt', 'isErased

In [5]:
query = """
    SELECT 'messages' AS table_name, COUNT(*) FROM messages
    UNION
    SELECT 'db_2.messages' AS table_name, COUNT(*) FROM db_2.messages
    UNION
    SELECT 'db_3.messages' AS table_name, COUNT(*) FROM db_3.messages
"""

cur.execute(query)
sum = 0
for count in cur.fetchall():
    sum += count['COUNT(*)']
print(str(sum) + " total messages.")

query = """
    SELECT COUNT(*) FROM (
        SELECT sent_at FROM messages
        UNION
        SELECT sent_at FROM db_2.messages
        UNION
        SELECT sent_at FROM db_3.messages
    ) AS derived;
"""

cur.execute(query)
unique = cur.fetchone()['COUNT(*)']
print(str(unique) + " unique messages.")

109222 total messages.
84340 unique messages.


In [14]:
query = """
WITH combined_messages AS (
    SELECT m.sent_at, m.rowid, m.id, m.json, m.readStatus, c.name AS conversationName, co.profileName, m.received_at, m.source, m.hasAttachments, m.hasFileAttachments, m.hasVisualMediaAttachments, m.type, m.body, m.isChangeCreatedByUs, m.isUserInitiatedMessage
    FROM messages AS m
    INNER JOIN conversations AS c ON m.conversationId = c.id
    INNER JOIN conversations AS co ON m.sourceUuid = co.uuid
    WHERE c.type = 'group'
    UNION
    SELECT m.sent_at, m.rowid, m.id, m.json, m.readStatus, c.name AS conversationName, co.profileName, m.received_at, m.source, m.hasAttachments, m.hasFileAttachments, m.hasVisualMediaAttachments, m.type, m.body, NULL AS isChangeCreatedByUs, NULL AS isUserInitiatedMessage
    FROM db_2.messages AS m
    INNER JOIN db_2.conversations AS c ON m.conversationId = c.id
    INNER JOIN db_2.conversations AS co ON m.sourceUuid = co.uuid
    WHERE c.type = 'group'
    UNION
    SELECT m.sent_at, m.rowid, m.id, m.json, m.readStatus, c.name AS conversationName, co.profileName, m.received_at, m.source, m.hasAttachments, m.hasFileAttachments, m.hasVisualMediaAttachments, m.type, m.body, m.isChangeCreatedByUs, m.isUserInitiatedMessage
    FROM db_3.messages AS m
    INNER JOIN db_3.conversations AS c ON m.conversationId = c.id
    INNER JOIN db_3.conversations AS co ON m.sourceUuid = co.uuid
    WHERE c.type = 'group'
)
SELECT *
FROM (
    SELECT cm.*,
        ROW_NUMBER() OVER (PARTITION BY cm.sent_at ORDER BY cm.sent_at) AS row_num
    FROM combined_messages AS cm
) AS numbered_messages
WHERE conversationName IS NOT NULL
    AND type IN ('incoming', 'outgoing')
    AND row_num = 1
ORDER BY sent_at;
"""

cur.execute(query)

In [ ]:
for i in range(10):
    row = cur.fetchone()
    display(dict(row))